<a href="https://colab.research.google.com/github/jkf87/Midm-2.0/blob/main/Midm_2_0_Mini_Instruct_gradio_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model_name = "K-intelligence/Midm-2.0-Mini-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
generation_config = GenerationConfig.from_pretrained(model_name)

prompt = "KT에 대해 소개해줘"

# message for inference
messages = [
    {"role": "system",
     "content": "Mi:dm(믿:음)은 KT에서 개발한 AI 기반 어시스턴트이다."},
    {"role": "user", "content": prompt}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

output = model.generate(
    input_ids.to("cuda"),
    generation_config=generation_config,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=128,
    do_sample=False,
)
print(tokenizer.decode(output[0]))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.61G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/10.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2024
Today Date: 04 Jul 2025

Mi:dm(믿:음)은 KT에서 개발한 AI 기반 어시스턴트이다. 너는 Mi:dm으로서 사용자에게 유용하고 안전한 응답을 제공해야 한다.

Mi:dm은 December 2024까지의 지식으로 학습되었으며 그 외의 지식을 묻는 경우에는 한계를 인정해야 한다.
오늘 날짜는 04 Jul 2025임을 참고하여 응답을 제공한다.

어시스턴트는 기본적으로 "한국어"를 사용한다. 사용자의 요청에 따라 생각하고 응답하는 언어는 달라질 수 있으며, 다른 요구사항이 없다면 입력 언어를 따라 응답하라.
코드 작성 시에는 요구되는 언어의 소스코드로 작성해야 하며, STEM(과학, 기술, 공학, 수학) 분야의 전문 용어는 원문을 그대로 유지하여 출력한다.

Mi:dm은 사용자 친화적으로 답변을 제공해야 한다. 사용자의 요청이 없다면 기본적으로 경어체를 사용해야 한다.
사용자의 요청에 따라 유용하고 관련성 있는 답변을 제공해야 한다. 이는 요청의 내용을 반영하여 이루어져야 한다.
특히, 사용자가 특정 답변의 형태를 요구한 경우 해당 형태를 반드시 준수하여 응답을 제공한다.(예. JSON, SQL, 코드 등) 그렇지 않다면 응답을 제공하기 가장 적절한 형태로 제공한다.
다양한 시각이 존재하는 주제에 대해서는 중립적인 입장을 유지해야 하지만, 한국 특화 인공지능 어시스턴트로서 정치적, 역사적, 외교적으로 한국인의 입장에서 응답을 생성해야 한다.
모호하거나 복잡한 요청으로 인해 답변 작성이 어려운 경우, 다시 한 번 생각해보고 추가정보를 요청해야 한다.

Mi:dm은 응답을 제공할 때 어시스턴트의 안전성 측면에서 다음 지침을 *반드시* 준수해야 한다.
- 비속어와 욕설을 사용하지 않아야 한다.
- 신뢰할 수 있는 응답을 생성하고, 전문영역에 대한 한계와 불확실성을 인정해야 

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import torch
import re

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

def clean_response(response):
    """응답에서 특별한 토큰들을 제거하고 정리"""
    # 특별한 토큰들 제거
    tokens_to_remove = [
        '<|begin_of_text|>', '<|start_header_id|>', '<|end_header_id|>',
        '<|eot_id|>', 'system', 'user', 'assistant'
    ]

    for token in tokens_to_remove:
        response = response.replace(token, '')

    # 연속된 공백 제거
    response = re.sub(r'\s+', ' ', response)

    # 시스템 메시지 부분 제거 (Cutting Knowledge Date부터 시작하는 부분)
    if 'Cutting Knowledge Date:' in response:
        parts = response.split('KT에 대해 소개해줘')
        if len(parts) > 1:
            response = parts[-1].strip()

    return response.strip()

def chat_with_model(message, history):
    # 시스템 메시지 (간단하게 설정)
    messages = [
        {"role": "system",
         "content": "Mi:dm(믿:음)은 KT에서 개발한 AI 기반 어시스턴트이다."}
    ]

    # 히스토리 추가
    for user_msg, bot_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": bot_msg})

    # 현재 메시지 추가
    messages.append({"role": "user", "content": message})

    # 토크나이징 및 생성
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    # attention mask 설정
    attention_mask = torch.ones_like(input_ids)

    # pad token 설정
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    with torch.no_grad():
        output = model.generate(
            input_ids.to(device),
            attention_mask=attention_mask.to(device),
            generation_config=generation_config,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=512,  # 더 길게 설정
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
        )

    # 응답 추출 및 정리
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # 입력 부분 제거
    input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    if input_text in response:
        response = response.replace(input_text, "").strip()

    # 응답 정리
    response = clean_response(response)

    # 빈 응답 처리
    if not response:
        response = "죄송합니다. 적절한 응답을 생성하지 못했습니다. 다시 시도해주세요."

    return response

# Gradio 인터페이스 생성
with gr.Blocks(theme=gr.themes.Soft(), title="Mi:dm AI 어시스턴트") as demo:
    gr.HTML("""
    <div style="text-align: center; margin-bottom: 20px;">
        <h1>🤖 Mi:dm (믿:음)</h1>
        <h3>KT AI 어시스턴트</h3>
        <p>KT에서 개발한 AI 기반 어시스턴트와 대화해보세요!</p>
    </div>
    """)

    chatbot = gr.Chatbot(
        height=500,
        show_label=False,
        container=True,
        show_copy_button=True
    )

    with gr.Row():
        msg = gr.Textbox(
            label="메시지",
            placeholder="안녕하세요! 무엇을 도와드릴까요?",
            lines=2,
            max_lines=10,
            scale=4
        )
        submit_btn = gr.Button("전송", variant="primary", scale=1)

    with gr.Row():
        clear_btn = gr.Button("대화 초기화", variant="secondary")

    # 예시 질문들
    with gr.Row():
        examples = [
            "안녕하세요! 자기소개 부탁드려요.",
            "KT에 대해 자세히 소개해주세요.",
            "KT의 주요 사업 영역은 무엇인가요?",
            "5G 기술에 대해 설명해주세요."
        ]

        for example in examples:
            gr.Button(example, size="sm").click(
                lambda x=example: (x, []),
                outputs=[msg, chatbot]
            )

    def respond(message, chat_history):
        if not message.strip():
            return chat_history, ""

        try:
            bot_response = chat_with_model(message, chat_history)
            chat_history.append([message, bot_response])
        except Exception as e:
            error_msg = f"오류가 발생했습니다: {str(e)}"
            chat_history.append([message, error_msg])

        return chat_history, ""

    def clear_chat():
        return []

    # 이벤트 연결
    submit_btn.click(
        respond,
        inputs=[msg, chatbot],
        outputs=[chatbot, msg]
    )

    msg.submit(
        respond,
        inputs=[msg, chatbot],
        outputs=[chatbot, msg]
    )

    clear_btn.click(
        clear_chat,
        outputs=[chatbot]
    )

# 실행
demo.launch(share=True, debug=True)

Using device: cuda
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3990cf34d56f46c62b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3990cf34d56f46c62b.gradio.live
